In [1]:
import json
import pandas as pd
import numpy as np
import requests
from sodapy import Socrata
from api_keys import MyAppToken

import dtale

from datetime import datetime, date, timedelta
import time

# Set to True if you want "all the info messages"
debug = False;
# print(MyAppToken)

#checking the day of the month and printing the result, this is used to filter the dataframe later
today = date.today()
day = today.strftime("%d")
                     
todaysDate = time.strftime("%m-%d-%Y")
todaysFile = todaysDate + ".csv"
                     
if(debug):
    print("Day =", day);

In [2]:
crime_data = "ijzp-q8t2"
client = Socrata("data.cityofchicago.org", MyAppToken)

In [3]:
def GetData(start_date, end_date):
    max_rows = 2000000;
    where_clause = "Date BETWEEN '" + start_date + "' AND '" + end_date + "'";
    city = "Chicago";
    
    df = pd.DataFrame(
        client.get(
            crime_data, 
            where=where_clause,
            limit=max_rows,
            exclude_system_fields=True
        )
    )
    client.close()

    df["city"] = city;

    df['day'] = pd.DatetimeIndex(df['date']).day
    df['month'] = pd.DatetimeIndex(df['date']).month
    df['year'] = pd.DatetimeIndex(df['date']).year
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')

    df["primary_type"] = df["primary_type"].str.lower().str.title()
    df["description"] = df["description"].str.lower().str.title()
    df["location"] = df["location_description"].str.lower().str.title()    

    # Organize: 
    dfReturn = df[[
        "city"
        , "primary_type"
        , "description"
        , "date"
        , "day"
        , "month"
        , "year"
        , "location"
        , "latitude"
        , "longitude"
        , "domestic"
    ]]
    
    return dfReturn;

In [4]:
df2020 = GetData('2020-01-01', '2020-04-07');
df2019 = GetData('2019-01-01', '2019-04-07');

In [5]:
df2019.head()

,city,primary_type,description,date,day,month,year,location,latitude,longitude,domestic
0,Chicago,Sex Offense,Aggravated Criminal Sexual Abuse,2019-01-01,1,1,2019,Residence,41.907072136,-87.731331357,False
1,Chicago,Offense Involving Children,Aggravated Criminal Sexual Abuse By Family Member,2019-01-01,1,1,2019,Apartment,41.853079711,-87.676856787,False
2,Chicago,Offense Involving Children,Child Abuse,2019-01-01,1,1,2019,Residence,41.875288057,-87.723999683,True
3,Chicago,Deceptive Practice,Financial Identity Theft Over $ 300,2019-01-01,1,1,2019,Apartment,41.929013312,-87.799805351,False
4,Chicago,Sex Offense,Agg Criminal Sexual Abuse,2019-01-01,1,1,2019,Apartment,41.896591951,-87.692793096,True


In [6]:
print(df2019.shape)
print(df2020.shape)

(62515, 11)
(55650, 11)


In [7]:
final = df2020.copy();
final = final.append(df2019);
print(final.shape)
final = final.sort_values(["date"])

(118165, 11)


In [8]:
final.sample(5)

,city,primary_type,description,date,day,month,year,location,latitude,longitude,domestic
12804,Chicago,Theft,Over $500,2019-01-19,19,1,2019,Residence,41.865154549,-87.713930253,False
40847,Chicago,Criminal Damage,To Property,2020-03-06,6,3,2020,Residence,41.750372342,-87.608843321,False
8093,Chicago,Theft,Over $500,2019-01-11,11,1,2019,Cta Platform,41.852866093,-87.631003027,False
46265,Chicago,Other Offense,Telephone Threat,2019-03-14,14,3,2019,Other,41.798030386,-87.629702378,True
43786,Chicago,Battery,Aggravated - Handgun,2020-03-11,11,3,2020,Street,41.682186258,-87.622791288,False


In [9]:
domestic_crimes = final['domestic'] == True
non_domestic_crimes = final['domestic'] == False
domestic_crimes_df = final[domestic_crimes]
non_domestic_crimes_df = final[non_domestic_crimes]

domestic_crimes_df.head()

,city,primary_type,description,date,day,month,year,location,latitude,longitude,domestic
669,Chicago,Assault,Aggravated - Handgun,2019-01-01,1,1,2019,Street,41.862528811,-87.703929853,True
677,Chicago,Battery,Domestic Battery Simple,2019-01-01,1,1,2019,Street,41.785468964,-87.724176235,True
683,Chicago,Battery,Domestic Battery Simple,2019-01-01,1,1,2019,Residence,41.881955638,-87.699986249,True
686,Chicago,Other Offense,Telephone Threat,2019-01-01,1,1,2019,Apartment,41.770565763,-87.585004532,True
692,Chicago,Theft,$500 And Under,2019-01-01,1,1,2019,Apartment,41.767869148,-87.587271934,True


In [10]:
#making dictionaries for mongoDB
non_domestic_chicago_data = non_domestic_crimes_df.to_dict()
domestic_chicago_data = domestic_crimes_df.to_dict()  

# print(non_domestic_chicago_data)
# print(domestic_chicago_data)

In [12]:
final.head()

,city,primary_type,description,date,day,month,year,location,latitude,longitude,domestic
666,Chicago,Weapons Violation,Unlawful Poss Of Handgun,2019-01-01,1,1,2019,Gas Station,41.793924612,-87.673580917,False
667,Chicago,Criminal Damage,To Vehicle,2019-01-01,1,1,2019,Auto / Boat / Rv Dealership,41.744337567,-87.652359252,False
668,Chicago,Theft,$500 And Under,2019-01-01,1,1,2019,Gas Station,41.8020092,-87.622063199,False
669,Chicago,Assault,Aggravated - Handgun,2019-01-01,1,1,2019,Street,41.862528811,-87.703929853,True
670,Chicago,Burglary,Forcible Entry,2019-01-01,1,1,2019,Residence,41.794352937,-87.734224585,False


In [26]:
aggs = final.groupby(
    ["date", "primary_type"]
).count()

aggs.reset_index();

aggs = aggs[["date, primary_type", "city"]]

KeyError: ('date, primary_type', 'city')

In [27]:
aggs

city  description  day  month  \
date       primary_type                                                       
2019-01-01 Assault                              63           63   63     63   
           Battery                             213          213  213    213   
           Burglary                             29           29   29     29   
           Crim Sexual Assault                  13           13   13     13   
           Criminal Damage                     124          124  124    124   
           Criminal Sexual Assault               2            2    2      2   
           Criminal Trespass                    16           16   16     16   
           Deceptive Practice                  116          116  116    116   
           Interference With Public Officer      7            7    7      7   
           Intimidation                          2            2    2      2   
           Kidnapping                            2            2    2      2   
           Liquor Law Violation                  2            2    2      2   
           Motor Vehicle Theft                  33           33   33     33   
           Narcotics                            36           36   36     36   
           Offense Involving Children           37           37   37     37   
           Other Offense                        76           76   76     76   
           Public Peace Violation                5            5    5      5   
           Robbery                              26           26   26     26   
           Sex Offense                          17           17   17     17   
           Theft                               152          152  152    152   
           Weapons Violation                    39           39   39     39   
2019-01-02 Arson                                 1            1    1      1   
           Assault                              43           43   43     43   
           Battery                             105          105  105    105   
           Burglary                             33           33   33     33   
           Concealed Carry License Violation     1            1    1      1   
           Crim Sexual Assault                   3            3    3      3   
           Criminal Damage                      61           61   61     61   
           Criminal Sexual Assault               2            2    2      2   
           Criminal Trespass                    16           16   16     16   
...                                            ...          ...  ...    ...   
2020-04-05 Criminal Sexual Assault               2            2    2      2   
           Criminal Trespass                    10           10   10     10   
           Deceptive Practice                   14           14   14     14   
           Homicide                              3            3    3      3   
           Motor Vehicle Theft                  19           19   19     19   
           Narcotics                             4            4    4      4   
           Offense Involving Children            1            1    1      1   
           Other Offense                        23           23   23     23   
           Public Peace Violation                3            3    3      3   
           Robbery                              20           20   20     20   
           Sex Offense                           1            1    1      1   
           Theft                                63           63   63     63   
           Weapons Violation                    17           17   17     17   
2020-04-06 Assault                              27           27   27     27   
           Battery                             104          104  104    104   
           Burglary                             19           19   19     19   
           Criminal Damage                      57           57   57     57   
           Criminal Trespass                     7            7    7      7   
           Decept

In [22]:
aggs.head(25)

city  description  day  month  \
date       primary_type                                                      
2019-01-01 Assault                             63           63   63     63   
           Battery                            213          213  213    213   
           Burglary                            29           29   29     29   
           Crim Sexual Assault                 13           13   13     13   
           Criminal Damage                    124          124  124    124   
           Criminal Sexual Assault              2            2    2      2   
           Criminal Trespass                   16           16   16     16   
           Deceptive Practice                 116          116  116    116   
           Interference With Public Officer     7            7    7      7   
           Intimidation                         2            2    2      2   
           Kidnapping                           2            2    2      2   
           Liquor Law Violation                 2            2    2      2   
           Motor Vehicle Theft                 33           33   33     33   
           Narcotics                           36           36   36     36   
           Offense Involving Children          37           37   37     37   
           Other Offense                       76           76   76     76   
           Public Peace Violation               5            5    5      5   
           Robbery                             26           26   26     26   
           Sex Offense                         17           17   17     17   
           Theft                              152          152  152    152   
           Weapons Violation                   39           39   39     39   
2019-01-02 Arson                                1            1    1      1   
           Assault                             43           43   43     43   
           Battery                            105          105  105    105   
           Burglary                            33           33   33     33   

                                             year  location  latitude  \
date       primary_type                                                 
2019-01-01 Assault                             63        63        63   
           Battery                            213       213       213   
           Burglary                            29        29        29   
           Crim Sexual Assault                 13        13        13   
           Criminal Damage                    124       124       123   
           Criminal Sexual Assault              2         2         2   
           Criminal Trespass                   16        16        16   
           Deceptive Practice                 116       105       112   
           Interference With Public Officer     7         7         7   
           Intimidation                         2         2         2   
           Kidnapping                           2         2         2   
           Liquor Law Violation                 2         2         2   
           Motor Vehicle Theft                 33        33        33   
           Narcotics                           36        36        36   
           Offense Involving Children          37        37        37   
           Other Offense                       76        76        76   
           Public Peace Violation               5         5         5   
           Robbery                             26        26        26   
           Sex Offense                         17        17        17   
           Theft                              152       152       149   
           Weapons Violation                   39        39        39   
2019-01-02 Arson                                1         1         1   
           Assault                             43        43        43   
           Battery                            105       105       105   
           Burglary                     

In [ ]:
aggs = final.groupby(["date", "primary_type"]).agg({
        'DaysOpen': [
            np.count_nonzero
            , np.mean
            , np.median
            , np.var
            , np.std
        ]
}).reset_index() # Gets rid of auto aggregation/hierarchy
aggs
# aggs = aggs.to_frame()
# aggs = aggs.reset_index(level=['ClosedMonth', 'Owner'])